In [11]:
"""

asset_pricing from quant-econ python
computes asset prices in an endowment economy when the endowment obeys
geometric growth driven by a finite state Markov chain. the transition
matrix of the markov chain is P, and the set of states is s. the
discount factor is beta, and gamma is the coefficient of relative risk
aversion in the household's utility function.

"""
from textwrap import dedent
import numpy as np
from numpy.linalg import solve

class AssetPrices(object):
    r"""
    a class to compute asset prices when the endowment follows a finite
    markov chain.
    
    parameters
    ----------
    beta: scalar, float
        discount factor
    P: array_like(float)
        tansition matrix
    s: array_like(float)
        growth rate of consumption
    gamma: scalar(float)
        coefficient of risk aversion
        
    attributes
    ----------
    beta, P, s, gamma: see aprameters
    n: scalar(int)
        the number of rows in P
    
    examples
    ---------
    omitted
    """
    def __init__(self, beta, P, s, gamma):
        self.beta, self.gamma=beta, gamma
        self.P, self.s=P, s
        self.n=self.P.shape[0]
        
    def __repr__(self):
        m="AssetPrices(beta={b:g}, P='{n:g} by {n:g}', s={s}, gamma={g:g})"
        return m.format(b=self.beta, n=self.P.shape[0], s=self.s, g=self.gamma)
    
    def __str__(self):
        m="""\
        AssetPrices(Merha and Prescott, 1985):
            -beta(discount factor):{b:g}
            -P(transition matrix): {n:g} by {n:g}
            -s(growth rate of consumption):{s:s}
            -gamma(coefficient of risk aversion):{g:g}
            """
        
        return dedent(m.format(b=self.beta, n=self.P.shape[0], s=repr(self.s), 
                              g=self.gamma))
    
    @property
    def P_tilde(self):
        P, s, gamma=self.P, self.s, self.gamma
        return P*s**(1.0-gamma)#using broadcasting
    
    @property
    def P_check(self):
        P, s, gamma=self.P, self.s, self.gamma
        return P*s**(-gamma)#using broadcasting
    
    def tree_price(self):
        """
        computes the function v such that the price of the lucas tree is
        v(lambda)C_t
        
        returns
        ----------
        v: array_like(float)
            lucas tree prices
            
        """
        
        #==simplify names==#
        beta=self.beta
        
        #==compute v==#
        P_tilde=self.P_tilde
        I=np.identity(self.n)
        O=np.ones(self.n)
        v=beta*solve(I-beta*P_tilde, P_tilde.dot(O))
        
        return v
    
    def consol_price(self, zeta):
        """
        computes price of a consol bond with payoff zeta
        
        patameters
        ----------
        zeta: scalar(float)
            coupon of the consol
        
        returns
        ----------
        p_bar: array_like(float)
            consol bond prices
            
        """
        #==simplify names==#
        beta=self.beta
        
        #==compute price==#
        P_check=self.P_check
        I=np.identity(self.n)
        O=np.ones(self.n)
        p_bar=beta*solve(I-beta*P_check, P_check.dot(zeta*O))
        
        return p_bar
    
    def call_option(self, zeta, p_s, T=[], epsilon=1e-8):
        """
        computes price of a call option on a consol bond, both finite
        and infinite horizon
        
        parameters
        ----------
        zeta: scalar(float)
            coupon of the consol
            
        p_s: scalar(float)
            strike price
        
        T: iterable(integers)
            length of option in the finite horizon case
        
        epsilon: scalar(float), optional(default=1e-8)
            tolerance for infinite horizon problem
            
        returns
        ----------
        w_bar: array_like(float)
            infinite horizon call option prices
            
        w_bars: dict
            a dictionary of key-value pairs {t:vec}; where t is one of
            the dates in the list T and vec is the option prices at that
            date
        """
        
        #==simplify names, initialize variables==#
        beta=self.beta
        P_check=self.P_check
        
        #==compute consol price==#
        v_bar=self.consol_price(zeta)
        
        #==compute option price==#
        w_bar=np.zeros(self.n)
        error=epsilon+1
        t=0
        w_bar= {}
        while error>epsilon:
            if t in T:
                w_bars[t]=w_bar
                
            #==maximize across columns==#
            to_stack = (beta*P_check.dot(w_bar), v_bar-p_s)
            w_bar_new=np.amax(np.vstack(to_stack), axis=0)
            
            #==find maximal difference of each componet==#
            error=np.amax(np.abs(w_bar-w_bar_new))
            
            #==update==#
            w_bar=w_bar_new
            t+=1
            
        return w_bar, w_bars

In [4]:
n=5
P=0.0125*np.ones((n,n))
P+=np.diag(0.95-0.0125*np.ones(5))
s=np.array([1.05, 1.025, 1.0, 0.975, 0.95])
gamma=2.0
beta=0.94
ap=AssetPrices(beta, P, s, gamma)
zeta=1.0
v=ap.tree_price()
print("lucas tree prices: %s" %v)

lucas tree prices: [ 12.72221763  14.72515002  17.57142236  21.93570661  29.47401578]


In [5]:
v_consol=ap.consol_price(zeta)
print("consol bond prices: %s" %v_consol)

consol bond prices: [  87.56860139  109.25108965  148.67554548  242.55144082  753.87100476]


In [12]:
p_s=150.0
w_bar, w_bars=ap.call_option(zeta, p_s, T=[10, 20, 30])
w_bar

TypeError: unsupported operand type(s) for *: 'float' and 'dict'